# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 11 2022 10:30AM,248343,10,8632675-SD,Eywa Pharma Inc.,Released
1,Oct 11 2022 10:28AM,248342,19,CLI00399.1,"CLINUVEL, Inc.",Released
2,Oct 11 2022 10:07AM,248340,19,60015989,"GUSA Granules USA, Inc.",Released
3,Oct 11 2022 10:05AM,248339,10,8555204,Eywa Pharma Inc.,Released
4,Oct 11 2022 10:05AM,248339,10,8555208,Eywa Pharma Inc.,Released
5,Oct 11 2022 10:05AM,248339,10,8550797-BO,Eywa Pharma Inc.,Released
6,Oct 11 2022 10:05AM,248339,10,8550799-BO,Eywa Pharma Inc.,Released
7,Oct 11 2022 10:05AM,248339,10,8550801-BO,Eywa Pharma Inc.,Released
8,Oct 11 2022 10:05AM,248339,10,8550803-BO,Eywa Pharma Inc.,Released
9,Oct 11 2022 10:05AM,248339,10,8550798-BO,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,248338,Released,1
29,248339,Released,30
30,248340,Released,1
31,248342,Released,1
32,248343,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
248338,NaN,NaN,1.0
248339,NaN,NaN,30.0
248340,NaN,NaN,1.0
248342,NaN,NaN,1.0
248343,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248034,0.0,0.0,1.0
248216,0.0,0.0,1.0
248260,26.0,0.0,1.0
248262,18.0,1.0,1.0
248264,0.0,0.0,30.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248034,0,0,1
248216,0,0,1
248260,26,0,1
248262,18,1,1
248264,0,0,30


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248034,0,0,1
1,248216,0,0,1
2,248260,26,0,1
3,248262,18,1,1
4,248264,0,0,30


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248034,,,1
1,248216,,,1
2,248260,26,,1
3,248262,18,1,1
4,248264,,,30


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 11 2022 10:30AM,248343,10,Eywa Pharma Inc.
1,Oct 11 2022 10:28AM,248342,19,"CLINUVEL, Inc."
2,Oct 11 2022 10:07AM,248340,19,"GUSA Granules USA, Inc."
3,Oct 11 2022 10:05AM,248339,10,Eywa Pharma Inc.
33,Oct 11 2022 9:57AM,248338,16,"SHL Pharma, LLC"
34,Oct 11 2022 9:57AM,248264,15,"Person & Covey, Inc."
64,Oct 11 2022 9:52AM,248268,15,"Alliance Pharma, Inc."
110,Oct 11 2022 9:51AM,248279,15,"Carwin Pharmaceutical Associates, LLC"
115,Oct 11 2022 9:33AM,248277,15,"Brookfield Pharmaceuticals, LLC"
150,Oct 11 2022 9:23AM,248334,10,"Lupin Research, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 11 2022 10:30AM,248343,10,Eywa Pharma Inc.,,,1
1,Oct 11 2022 10:28AM,248342,19,"CLINUVEL, Inc.",,,1
2,Oct 11 2022 10:07AM,248340,19,"GUSA Granules USA, Inc.",,,1
3,Oct 11 2022 10:05AM,248339,10,Eywa Pharma Inc.,,,30
4,Oct 11 2022 9:57AM,248338,16,"SHL Pharma, LLC",,,1
5,Oct 11 2022 9:57AM,248264,15,"Person & Covey, Inc.",,,30
6,Oct 11 2022 9:52AM,248268,15,"Alliance Pharma, Inc.",,,46
7,Oct 11 2022 9:51AM,248279,15,"Carwin Pharmaceutical Associates, LLC",,,5
8,Oct 11 2022 9:33AM,248277,15,"Brookfield Pharmaceuticals, LLC",,,35
9,Oct 11 2022 9:23AM,248334,10,"Lupin Research, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 11 2022 10:30AM,248343,10,Eywa Pharma Inc.,1,,
1,Oct 11 2022 10:28AM,248342,19,"CLINUVEL, Inc.",1,,
2,Oct 11 2022 10:07AM,248340,19,"GUSA Granules USA, Inc.",1,,
3,Oct 11 2022 10:05AM,248339,10,Eywa Pharma Inc.,30,,
4,Oct 11 2022 9:57AM,248338,16,"SHL Pharma, LLC",1,,
5,Oct 11 2022 9:57AM,248264,15,"Person & Covey, Inc.",30,,
6,Oct 11 2022 9:52AM,248268,15,"Alliance Pharma, Inc.",46,,
7,Oct 11 2022 9:51AM,248279,15,"Carwin Pharmaceutical Associates, LLC",5,,
8,Oct 11 2022 9:33AM,248277,15,"Brookfield Pharmaceuticals, LLC",35,,
9,Oct 11 2022 9:23AM,248334,10,"Lupin Research, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 11 2022 10:30AM,248343,10,Eywa Pharma Inc.,1,,
1,Oct 11 2022 10:28AM,248342,19,"CLINUVEL, Inc.",1,,
2,Oct 11 2022 10:07AM,248340,19,"GUSA Granules USA, Inc.",1,,
3,Oct 11 2022 10:05AM,248339,10,Eywa Pharma Inc.,30,,
4,Oct 11 2022 9:57AM,248338,16,"SHL Pharma, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 11 2022 10:30AM,248343,10,Eywa Pharma Inc.,1.0,NaN,NaN
1,Oct 11 2022 10:28AM,248342,19,"CLINUVEL, Inc.",1.0,NaN,NaN
2,Oct 11 2022 10:07AM,248340,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
3,Oct 11 2022 10:05AM,248339,10,Eywa Pharma Inc.,30.0,NaN,NaN
4,Oct 11 2022 9:57AM,248338,16,"SHL Pharma, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 11 2022 10:30AM,248343,10,Eywa Pharma Inc.,1.0,0.0,0.0
1,Oct 11 2022 10:28AM,248342,19,"CLINUVEL, Inc.",1.0,0.0,0.0
2,Oct 11 2022 10:07AM,248340,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
3,Oct 11 2022 10:05AM,248339,10,Eywa Pharma Inc.,30.0,0.0,0.0
4,Oct 11 2022 9:57AM,248338,16,"SHL Pharma, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1738290,99.0,20.0,5.0
15,1489610,118.0,1.0,44.0
16,496654,2.0,0.0,0.0
18,248317,1.0,0.0,0.0
19,993323,2.0,2.0,0.0
20,744958,49.0,34.0,0.0
21,744521,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1738290,99.0,20.0,5.0
1,15,1489610,118.0,1.0,44.0
2,16,496654,2.0,0.0,0.0
3,18,248317,1.0,0.0,0.0
4,19,993323,2.0,2.0,0.0
5,20,744958,49.0,34.0,0.0
6,21,744521,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,99.0,20.0,5.0
1,15,118.0,1.0,44.0
2,16,2.0,0.0,0.0
3,18,1.0,0.0,0.0
4,19,2.0,2.0,0.0
5,20,49.0,34.0,0.0
6,21,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,99.0
1,15,Released,118.0
2,16,Released,2.0
3,18,Released,1.0
4,19,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21
Status,,,,,,,
Completed,5.0,44.0,0.0,0.0,0.0,0.0,0.0
Executing,20.0,1.0,0.0,0.0,2.0,34.0,0.0
Released,99.0,118.0,2.0,1.0,2.0,49.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21
0,Completed,5.0,44.0,0.0,0.0,0.0,0.0,0.0
1,Executing,20.0,1.0,0.0,0.0,2.0,34.0,0.0
2,Released,99.0,118.0,2.0,1.0,2.0,49.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21
0,Completed,5.0,44.0,0.0,0.0,0.0,0.0,0.0
1,Executing,20.0,1.0,0.0,0.0,2.0,34.0,0.0
2,Released,99.0,118.0,2.0,1.0,2.0,49.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()